In [12]:
# -*- coding: utf-8 -*-
# @Time : 2021/4/15 15:17
# @Author : Jclian91
# @File : keras_cross_entropy_test.ipynb
# @Place : Yangpu, Shanghai
# import related modules
import math
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.python import math_ops, array_ops
from keras import backend as K
from keras.losses import (mean_squared_error,
                          mean_absolute_error,
                          binary_crossentropy,
                          categorical_crossentropy,
                          sparse_categorical_crossentropy,
                          hinge
                          )

# filter warning message
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
# mse
y_true = tf.constant([[10.0, 7.0]])
y_pred = tf.constant([[8.0, 6.0]])
loss = mean_squared_error(y_true, y_pred)
loss = K.eval(loss)
print(f'Value of Mean Squared Error is {loss}')

Value of Mean Squared Error is [2.5]


In [4]:
# mae
y_true = tf.constant([[10.0, 7.0]])
y_pred = tf.constant([[8.0, 6.0]])
loss = mean_absolute_error(y_true, y_pred)
loss = K.eval(loss)
print(f'Value of Mean Absolute Error is {loss}')

Value of Mean Absolute Error is [1.5]


In [13]:
# Binary Cross Entropy
y_true = tf.constant([[1.0], [0.0]])
y_pred = tf.constant([[0.9], [0.2]])
loss = binary_crossentropy(y_true, y_pred)
loss = K.eval(loss)
python_loss = [1.0 * (-math.log(0.9)), 1.0 * (-math.log(0.8))]
print(f'Compute Binary Cross Entropy in python: {python_loss}')
print(f'Compute Binary Cross Entropy in Keras: {loss}')

Compute Binary Cross Entropy in python: [0.10536051565782628, 0.2231435513142097]
Compute Binary Cross Entropy in Keras: [0.10536053 0.22314355]


In [6]:
# Categorical Cross Entropy
y_true = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]
y_pred = [[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]]
loss = (-math.log(0.9)-math.log(0.89)-math.log(0.94))/3
print(f"compute CCE by python: {loss}")
loss = log_loss(y_true, y_pred)
print(f"compute CCE by sklearn: {loss}")
loss = K.sum(categorical_crossentropy(tf.constant(y_true), tf.constant(y_pred)))/3
loss = K.eval(loss)
print(f"compute CCE by Keras: {loss}")

compute CCE by python: 0.09458991187728844
compute CCE by sklearn: 0.09458991187728844
compute CCE by Keras: 0.09458992630243301


In [7]:
# Sparse Categorical Cross Entropy
t = LabelEncoder()
y_pred = tf.constant([[0.1, 0.1, 0.8], [0.1, 0.4, 0.5], [0.5, 0.3, 0.2], [0.6, 0.3, 0.1]])
y_true = t.fit_transform(['Rain', 'Rain', 'High Changes of Rain', 'No Rain'])
print("transformed label: ", y_true)
y_true = tf.constant(y_true)
loss = sparse_categorical_crossentropy(y_true, y_pred)
loss = K.eval(loss)
print(f'Value of Sparse Categorical Cross Entropy is ', loss)

transformed label:  [2 2 0 1]
Value of Sparse Categorical Cross Entropy is  [0.22314355 0.6931472  0.6931471  1.2039728 ]


In [8]:
# hinge loss
y_true = tf.constant([[0., 1.], [0., 0.]])
y_pred = tf.constant([[0.7, 0.3], [0.4, 0.6]])
loss = hinge(y_true, y_pred)
a = K.eval(loss)
print("hinge loss: ", a)

hinge loss:  [0.85 1.  ]


In [9]:
# Custom Loss Function: categorical_crossentropy_with_label_smoothing
def categorical_crossentropy_with_label_smoothing(y_true, y_pred, label_smoothing=0.1):
    num_classes = math_ops.cast(array_ops.shape(y_true)[1], y_pred.dtype)
    y_true = y_true * (1.0 - label_smoothing) + (label_smoothing / num_classes)
    return categorical_crossentropy(y_true, y_pred)


y_true = tf.constant([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]])
y_pred = tf.constant([[.9, .05, .05], [.05, .89, .06], [.05, .01, .94]])
loss = categorical_crossentropy_with_label_smoothing(y_true, y_pred, label_smoothing=0.3)
loss = K.eval(loss)
print(f"compute CCE with label smoothing by Custom Loss function in Keras: {loss}")
print("compute CCE with label smoothing by Custom Loss function in Keras: {}, {}, {}\n".format(
0.8*(-math.log(0.9))+0.1*(-math.log(0.05))+0.1*(-math.log(0.05)),
0.1*(-math.log(0.05))+0.8*(-math.log(0.89))+0.1*(-math.log(0.06)),
0.1*(-math.log(0.05))+0.1*(-math.log(0.01))+0.8*(-math.log(0.94))
))

compute CCE with label smoothing by Custom Loss function in Keras: [0.68343496 0.6741414  0.8095906 ]
compute CCE with label smoothing by Custom Loss function in Keras: 0.6834348672370592, 0.6741413520361639, 0.8095905689286783



In [10]:
# use Custom Loss Function: categorical_crossentropy_with_label_smoothing for classification model in Keras
import keras as K
import pandas as pd
# 1. 读取数据
df = pd.read_csv("iris.csv", header=None)
targets = df[4].unique()
targets_dict = dict(zip(targets, range(len(targets))))
df["target"] = df[4].apply(lambda x: targets_dict[x])
print(targets_dict)
print(df.head())
train_df = df.sample(frac=0.8)
test_df = df.drop(train_df.index)

# 2. 定义模型
init = K.initializers.glorot_uniform(seed=1)
simple_adam = K.optimizers.Adam()
model = K.models.Sequential()
model.add(K.layers.Dense(units=5, input_dim=4, kernel_initializer=init, activation='relu'))
model.add(K.layers.Dense(units=6, kernel_initializer=init, activation='relu'))
model.add(K.layers.Dense(units=3, kernel_initializer=init, activation='softmax'))
model.compile(loss=categorical_crossentropy_with_label_smoothing, optimizer=simple_adam, metrics=['accuracy'])
# 3. 模型训练
train_x = train_df[[0, 1, 2, 3]]
train_y = train_df["target"]
test_x = test_df[[0, 1, 2, 3]]
test_y = test_df["target"]
h = model.fit(train_x, train_y, batch_size=1, epochs=100, shuffle=True, verbose=1)
# 4. 评估模型
eval = model.evaluate(test_x, test_y, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100))

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
     0    1    2    3            4  target
0  5.1  3.5  1.4  0.2  Iris-setosa       0
1  4.9  3.0  1.4  0.2  Iris-setosa       0
2  4.7  3.2  1.3  0.2  Iris-setosa       0
3  4.6  3.1  1.5  0.2  Iris-setosa       0
4  5.0  3.6  1.4  0.2  Iris-setosa       0
Epoch 1/100
120/120 [==============================] - 0s 2ms/step - loss: 3.4442 - acc: 0.1833
Epoch 2/100
120/120 [==============================] - 0s 623us/step - loss: 3.4441 - acc: 0.1333
Epoch 3/100
120/120 [==============================] - 0s 665us/step - loss: 3.4441 - acc: 0.2750
Epoch 4/100
120/120 [==============================] - 0s 615us/step - loss: 3.4441 - acc: 0.1750
Epoch 5/100
120/120 [==============================] - 0s 623us/step - loss: 3.4441 - acc: 0.2000
Epoch 6/100
120/120 [==============================] - 0s 607us/step - loss: 3.4441 - acc: 0.4083
Epoch 7/100
120/120 [==============================] - 0s 733us/step - loss: 3.4441 - acc: 0.3

120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 82/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 83/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 84/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 85/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 86/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 87/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 88/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 89/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 90/100
120/120 [==============================] - 0s 1ms/step - loss: 3.4441 - acc: 1.0000
Epoch 91/100
120/120 [=====================